In [2]:
import xlwings as xw
import pandas as pd
import numpy as np
from datetime import (
    datetime as dt,
)
from openpyxl.utils import (
    get_column_letter as gcl,
)

## Automating Book Selections from Open Books

#### Demonstrative DT Function

In [169]:
dt.strptime('2019 January', '%Y %B') > dt.strptime('2019 December', '%Y %B')

False

In [170]:
afv_books = list(b.name for b in xw.books if 'US Alt-Fuel' in b.name)

In [171]:
afv_books

['US Alt-Fuel Sales History To Date 2019 December.xlsx',
 'US Alt-Fuel Sales History To Date 2019 November.xlsx']

#### last two words from book name minus '.xlsx' sliced out

In [172]:
book_dates = list(b[:-5].split()[-2:] for b in afv_books)
book_dates

[['2019', 'December'], ['2019', 'November']]

#### Flattened list object

In [173]:
book_dates = [' '.join(bd) for bd in book_dates]
book_dates

['2019 December', '2019 November']

#### DT.strptime, tupled with string then sorted from small to large

In [174]:
bookDT = [(dt.strptime(bd, '%Y %B').date(), bd) for bd in book_dates]
bookDT.sort()
bookDT

[(datetime.date(2019, 11, 1), '2019 November'),
 (datetime.date(2019, 12, 1), '2019 December')]

In [175]:
o = bookDT[0][1]
n = bookDT[1][1]

In [176]:
o

'2019 November'

In [177]:
n

'2019 December'

In [178]:
old = xw.books[[b for b in afv_books if o in b][0]]
new = xw.books[[b for b in afv_books if n in b][0]]

### Two Book Objects Created

In [179]:
old.name

'US Alt-Fuel Sales History To Date 2019 November.xlsx'

In [180]:
new.name

'US Alt-Fuel Sales History To Date 2019 December.xlsx'

### Slicing out Sheets

In [181]:
list(enumerate(s.name for s in old.sheets))

[(0, 'Fuel Cell Vehicles'),
 (1, 'Hybrid Vehicles'),
 (2, 'Plug-in Hybrid'),
 (3, 'Electric Vehicles'),
 (4, 'Annual Totals'),
 (5, 'Charting'),
 (6, 'PPR Charting'),
 (7, 'Monthly Sales (+ Registrations)'),
 (8, 'Monthly Sales Recap'),
 (9, 'Sales v Obj')]

In [182]:
list(enumerate(s.name for s in new.sheets))

[(0, 'Fuel Cell Vehicles'),
 (1, 'Hybrid Vehicles'),
 (2, 'Plug-in Hybrid'),
 (3, 'Electric Vehicles'),
 (4, 'Annual Totals'),
 (5, 'Charting'),
 (6, 'PPR Charting'),
 (7, 'Monthly Sales (+ Registrations)'),
 (8, 'Monthly Sales Recap'),
 (9, 'Sales v Obj')]

In [183]:
fcv_o = old.sheets['Fuel Cell Vehicles']
hev_o = old.sheets['Hybrid Vehicles']
phev_o = old.sheets['Plug-in Hybrid']
bev_o = old.sheets['Electric Vehicles']
all_o = old.sheets['Annual Totals']

In [184]:
fcv_n = new.sheets['Fuel Cell Vehicles']
hev_n = new.sheets['Hybrid Vehicles']
phev_n = new.sheets['Plug-in Hybrid']
bev_n = new.sheets['Electric Vehicles']
all_n = new.sheets['Annual Totals']

## Comparing DataFrames

In [185]:
old_df = pd.DataFrame(all_o['A4:ZA4'].value, columns=['old4'])
old_df['old_col'] = old_df.index + 1
old_df['old_col'] = old_df['old_col'].apply(lambda x: col_letter(x))
old_df

,old4,old_col
0,Year,A
1,Industry Sales,B
2,Alt-Fuel Vehicle Sales,C
3,CA,D
4,NE,E
...,...,...
672,None,YW
673,None,YX
674,None,YY
675,None,YZ


In [186]:
new_df = pd.DataFrame(all_n['A4:ZA4'].value, columns=['new4'])
new_df['new_col'] = new_df.index + 1
new_df['new_col'] = new_df['new_col'].apply(lambda x: col_letter(x))
new_df

,new4,new_col
0,Year,A
1,Industry Sales,B
2,Alt-Fuel Vehicle Sales,C
3,CA,D
4,NE,E
...,...,...
672,None,YW
673,None,YX
674,None,YY
675,None,YZ


In [187]:
joined = pd.concat([old_df, new_df], axis=1)
joined.drop_duplicates()
joined

,old4,old_col,new4,new_col
0,Year,A,Year,A
1,Industry Sales,B,Industry Sales,B
2,Alt-Fuel Vehicle Sales,C,Alt-Fuel Vehicle Sales,C
3,CA,D,CA,D
4,NE,E,NE,E
...,...,...,...,...
672,None,YW,None,YW
673,None,YX,None,YX
674,None,YY,None,YY
675,None,YZ,None,YZ


In [194]:
joined[joined['old4'] == joined['new4']]

,old4,old_col,new4,new_col
0,Year,A,Year,A
1,Industry Sales,B,Industry Sales,B
2,Alt-Fuel Vehicle Sales,C,Alt-Fuel Vehicle Sales,C
3,CA,D,CA,D
4,NE,E,NE,E
...,...,...,...,...
101,Acura ILX Hybrid,CX,Acura ILX Hybrid,CX
102,Acura RLX Hybrid,CY,Acura RLX Hybrid,CY
103,Acura MDX Hybrid,CZ,Acura MDX Hybrid,CZ
104,Ford C-Max Hybrid,DA,Ford C-Max Hybrid,DA


In [213]:
joined[(joined['old4'] != joined['new4']) & (joined['new4'].notna())]

,old4,old_col,new4,new_col
106,Mercury Mariner Hybrid,DC,Ford Explorer,DC
107,Mercury Milan Hybrid*,DD,Mercury Mariner Hybrid,DD
108,Ford Fusion Hybrid,DE,Mercury Milan Hybrid*,DE
109,Mazda Tribute Hybrid*,DF,Ford Fusion Hybrid,DF
110,Lincoln MKZ Hybrid,DG,Mazda Tribute Hybrid*,DG
...,...,...,...,...
259,None,IZ,Volvo AFV Share,IZ
260,None,JA,Fiat-Chrysler AFV Share,JA
261,None,JB,Mitsubishi AFV Share,JB
262,None,JC,Jaguar AFV Share,JC


In [209]:
joined[joined['new4'] == None]

,old4,old_col,new4,new_col


In [59]:
from_new = list(zip(list(new_df[0]), list(new_df['Col'])))
from_new

[('Year', 1),
 ('Industry Sales', 2),
 ('Alt-Fuel Vehicle Sales', 3),
 ('CA', 4),
 ('NE', 5),
 ('Fleet (Reference)', 6),
 ('Total US', 7),
 ('Honda Clarity FCV', 8),
 ('Hyundai Tucson FCV', 9),
 ('Hyundai Nexo', 10),
 (None, 11),
 (None, 12),
 (None, 13),
 (None, 14),
 ('Toyota Mirai', 15),
 ('Honda Clarity', 16),
 ('Hyundai Tucson', 17),
 ('Non-Fleet', 18),
 ('Fleet', 19),
 ('Total US (no Hawaii)', 20),
 ('Hawaii', 21),
 ('Total US Prius', 22),
 ('Non-Fleet', 23),
 ('Fleet', 24),
 ('Total US (no Hawaii)', 25),
 ('Hawaii', 26),
 ('Total US Prius v', 27),
 ('Non-Fleet', 28),
 ('Fleet', 29),
 ('Total US (no Hawaii)', 30),
 ('Hawaii', 31),
 ('Total US Prius c', 32),
 ('Non-Fleet', 33),
 ('Fleet', 34),
 ('Total US (no Hawaii)', 35),
 ('Hawaii', 36),
 ('Total US Prius Family', 37),
 ('Non-Fleet', 38),
 ('Fleet', 39),
 ('Total US (no Hawaii)', 40),
 ('Total US Avalon Hybrid', 41),
 ('Non-Fleet', 42),
 ('Fleet', 43),
 ('Total US (no Hawaii)', 44),
 ('Total US Camry Hybrid', 45),
 ('Non-Fleet'

In [60]:
from_old = list(zip(list(old_df[0]), list(old_df['Col'])))
from_old

[('Year', 1),
 ('Industry Sales', 2),
 ('Alt-Fuel Vehicle Sales', 3),
 ('CA', 4),
 ('NE', 5),
 ('Fleet (Reference)', 6),
 ('Total US', 7),
 ('Honda Clarity FCV', 8),
 ('Hyundai Tucson FCV', 9),
 ('Hyundai Nexo', 10),
 (None, 11),
 (None, 12),
 (None, 13),
 (None, 14),
 ('Toyota Mirai', 15),
 ('Honda Clarity', 16),
 ('Hyundai Tucson', 17),
 ('Non-Fleet', 18),
 ('Fleet', 19),
 ('Total US (no Hawaii)', 20),
 ('Hawaii', 21),
 ('Total US Prius', 22),
 ('Non-Fleet', 23),
 ('Fleet', 24),
 ('Total US (no Hawaii)', 25),
 ('Hawaii', 26),
 ('Total US Prius v', 27),
 ('Non-Fleet', 28),
 ('Fleet', 29),
 ('Total US (no Hawaii)', 30),
 ('Hawaii', 31),
 ('Total US Prius c', 32),
 ('Non-Fleet', 33),
 ('Fleet', 34),
 ('Total US (no Hawaii)', 35),
 ('Hawaii', 36),
 ('Total US Prius Family', 37),
 ('Non-Fleet', 38),
 ('Fleet', 39),
 ('Total US (no Hawaii)', 40),
 ('Total US Avalon Hybrid', 41),
 ('Non-Fleet', 42),
 ('Fleet', 43),
 ('Total US (no Hawaii)', 44),
 ('Total US Camry Hybrid', 45),
 ('Non-Fleet'

In [65]:
list(set(from_new) - set(from_old))

[('MB GLC 350e Plug In', 168),
 ('BMW X5 XDrive40E', 172),
 ('Kia Niro', 143),
 ('Nissan Altima Hybrid', 125),
 ('MB B-Class Hybrid B250 e', 202),
 ('Subaru Crosstrek', 191),
 ('MB E-Class Hybrid', 132),
 ('Hyundai Kona', 213),
 ('BMW Combined Alt-Fuel Sales', 241),
 (None, 197),
 ('Honda Clarity EV', 201),
 ('Volvo XC90 Hybrid', 184),
 ('Hyundai Ioniq PHEV', 180),
 ('Lincoln MKZ Hybrid', 112),
 ('Chevy Malibu Hybrid/Eco', 119),
 ('MB ML Hybrid', 133),
 ('Nissan Combined Alt-Fuel Sales', 239),
 ('Kia Soul EV', 211),
 ('Nissan  AFV Share', 255),
 ('Chevy Tahoe Hybrid*', 118),
 ('Mitsubishi Outlander Plug-in', 189),
 ('BMW i3', 204),
 ('MB GLE 550 Plug-in Hybrid', 167),
 (None, 198),
 ('Mercury Mariner Hybrid', 108),
 ('Ford Fusion Hybrid', 110),
 ('Land Rover Range Rover Sport PHEV', 193),
 ('Volvo XC60 PHEV', 185),
 ('Mercury Milan Hybrid*', 109),
 ('BMW Active Hybrid 3', 135),
 ('Hyundai Sonata Hybrid', 140),
 ('Toyota AFV Total', 226),
 ('Mitsubishi Combined Alt-Fuel Sales', 246),
 (

In [47]:
list(set(new_df[0]) - set(old_df[0]))

['Ford Explorer',
 'Land Rover Range Rover Sport PHEV',
 'Lincoln Aviator PHEV',
 'Land Rover Range Rover PHEV',
 'Subaru Crosstrek']

In [49]:
list(set(all_o['A4:ZA4'].value) - set(all_n['A4:ZA4'].value))

[]

In [26]:
set(all_o['A4:ZA4'].value) - set(all_n['A4:ZA4'].value)

set()